In [2]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from torch.utils.data import TensorDataset, DataLoader
import re
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
from razdel import tokenize
from navec import Navec

In [3]:
device = torch.device ( "cuda:0" if torch.cuda.is_available() else "cpu" )
print ( device )

cuda:0


In [4]:
df = pd.read_csv("../datasets/Petitions.csv")
corpus = df.sample(30000, random_state=42)
corpus

,id,public_petition_text,reason_category
1638,3315301,На газоне разбросан различный мусор. \nПросьба...,Благоустройство
53659,3372310,"Конструкции, препятствующие парковке",Благоустройство
25910,3294518,Лестницу убирают раз в три месяца и просто выл...,Содержание МКД
52453,3158213,мусор с задней стороны дома,Благоустройство
12590,3356661,Снова не работает уличный фонарь у входной две...,Содержание МКД
...,...,...,...
33494,3270830,"Мусор на внутридомовой территории\r\nБутылки, ...",Благоустройство
38310,3044011,Не работает фонарь на Красносельской около пер...,Благоустройство
57588,3133497,Необходимо очистить дорожный знак от объявлений,Благоустройство
51045,3080984,Приведите пожалуйста информационный стенд в по...,Содержание МКД


In [5]:
corpus_text = corpus["public_petition_text"]
corpus_text_category = corpus["reason_category"]

In [6]:
print(corpus_text)
print(type(corpus_text))

1638     На газоне разбросан различный мусор. \nПросьба...
53659                 Конструкции, препятствующие парковке
25910    Лестницу убирают раз в три месяца и просто выл...
52453                          мусор с задней стороны дома
12590    Снова не работает уличный фонарь у входной две...
                               ...                        
33494    Мусор на внутридомовой территории\r\nБутылки, ...
38310    Не работает фонарь на Красносельской около пер...
57588      Необходимо очистить дорожный знак от объявлений
51045    Приведите пожалуйста информационный стенд в по...
44526    не законное использование общедомового имущест...
Name: public_petition_text, Length: 30000, dtype: object
<class 'pandas.core.series.Series'>


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()
corpus_text_category_encoded = le.fit_transform(corpus_text_category)
dict(zip(range(0, len(set(corpus_text_category_encoded))),le.inverse_transform(list(set(corpus_text_category_encoded)))))

{0: 'Благоустройство',
 1: 'Водоотведение',
 2: 'Водоснабжение',
 3: 'Кровля',
 4: 'Нарушение порядка пользования общим имуществом',
 5: 'Нарушение правил пользования общим имуществом',
 6: 'Незаконная информационная и (или) рекламная конструкция',
 7: 'Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)',
 8: 'Повреждения или неисправность элементов уличной инфраструктуры',
 9: 'Подвалы',
 10: 'Санитарное состояние',
 11: 'Содержание МКД',
 12: 'Состояние рекламных или информационных конструкций',
 13: 'Фасад',
 14: 'Центральное отопление'}

In [9]:
corpus_encoded = []
for i in range(len(corpus_text)):
    corpus_encoded.append((corpus_text.iloc[i], corpus_text_category_encoded[i]))

In [10]:
navec = Navec.load('../models/navec_hudlit_v1_12B_500K_300d_100q.tar')

In [11]:
from nltk.corpus import stopwords
def prepare_data(corpus):
    morph = MorphAnalyzer()
    from nltk.corpus import stopwords
    stopwords = stopwords.words('russian')
    prepared_data = []
    for petition, label in corpus:   
        data = petition.lower()
        data1 = re.sub('[\\r|\\n]+', ' ', data)
        data2 = re.sub('[a-zA-Z]+', '', data1)
        data3 = re.sub('[0-9]+', '', data2)
        data4 = re.sub('[^\s^\w]+', '', data3)
        data5 = list(tokenize(data4))
        data6 = map(lambda x: x.text , data5)
        data7 = map(lambda x: morph.normal_forms(x)[0], data6)
        data8 = [w for w in data7 if w not in stopwords]
        data9 = [navec[x].tolist() for x in data8 if x in navec]
        if(len(data9)):
            prepared_data.append((torch.tensor(data9).type(torch.float32), label))
    return prepared_data

In [12]:
corpus_text_prep = prepare_data(corpus_encoded)

In [13]:
from torch.nn.utils.rnn import pad_sequence
corpus_text_padded = pad_sequence([tupl[0] for tupl in corpus_text_prep], True)

In [14]:
corpus_text_padded.shape

torch.Size([29904, 229, 300])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(corpus_text_padded, [tupl[1] for tupl in corpus_text_prep], test_size=0.1, stratify=[tupl[1] for tupl in corpus_text_prep])


In [16]:
ds = TensorDataset(X_train, torch.tensor(y_train).type(torch.int64))
dl = DataLoader(ds, batch_size=32, shuffle=True)

In [24]:
class Rnn(nn.Module):
    def __init__(self):
        super(Rnn, self).__init__()
        self.rnn = nn.RNN(300, 128, batch_first=True)
        self.first_linear = nn.Linear(128, 15)
    def forward(self, x):
        y = self.rnn(x)[0]
        #y = self.first_linear(torch.index_select(y, 1, torch.tensor(y.shape[1]-1).to(device)))
        y = self.first_linear(y.mean(dim=1))
        return y

In [25]:
loss = nn.CrossEntropyLoss()
model = Rnn()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [26]:
epochs = 10
for epoch in range(epochs):
    sum_loss = 0
    for x_b, y_b in dl:
        x_b = x_b.to(device)
        y_b = y_b.to(device)
        optimizer.zero_grad()
        outputs = model(x_b)
        #outputs= outputs.view(x_b.shape[0],15)
        loss_value = loss(outputs, y_b)
        sum_loss += loss_value
        loss_value.backward()
        optimizer.step()
        x_b = x_b.to("cpu")
        y_b = y_b.to("cpu")
        
        #sys.stderr.write(f'Батч {batch + 1}/{len(dl)}, Значение функции потерь: {loss_value.item()}\r')

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {sum_loss/len(dl)}')

Эпоха 1, Значение функции потерь: 1.221165418624878
Эпоха 2, Значение функции потерь: 1.092416524887085
Эпоха 3, Значение функции потерь: 1.1563538312911987
Эпоха 4, Значение функции потерь: 1.2457013130187988
Эпоха 5, Значение функции потерь: 1.2671200037002563
Эпоха 6, Значение функции потерь: 1.1804628372192383
Эпоха 7, Значение функции потерь: 1.1972708702087402
Эпоха 8, Значение функции потерь: 1.1443296670913696
Эпоха 9, Значение функции потерь: 1.125468134880066
Эпоха 10, Значение функции потерь: 1.1553950309753418


In [27]:
model.to("cpu")
answ = model.forward(X_test)

In [28]:
answ_encoded = []
for i in answ:
    answ_encoded.append(torch.argmax(i).to("cpu").item())

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, answ_encoded))

              precision    recall  f1-score   support

           0       0.68      0.98      0.80      1748
           1       0.00      0.00      0.00        11
           2       0.00      0.00      0.00        40
           3       0.00      0.00      0.00        41
           4       0.00      0.00      0.00        12
           5       0.00      0.00      0.00       102
           6       0.00      0.00      0.00        92
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        56
           9       0.00      0.00      0.00        11
          10       0.00      0.00      0.00        22
          11       0.73      0.46      0.57       726
          12       0.00      0.00      0.00        32
          13       0.00      0.00      0.00        72
          14       0.00      0.00      0.00        15

    accuracy                           0.68      2991
   macro avg       0.09      0.10      0.09      2991
weighted avg       0.57   

c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [30]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.rnn = nn.LSTM(300, 64, batch_first = True)
        self.first_linear = nn.Linear(64, 15)
        #self.first_linear = nn.Linear(64, 32)
        #self.first_activ = nn.LeakyReLU()
        #self.second_linear = nn.Linear(32, 15)
    def forward(self, x):
        y = self.rnn(x)[0]
        #y = self.first_linear(torch.index_select(y, 1, torch.tensor(y.shape[1]-1).to(device)))
        y = self.first_linear(y.mean(dim=1))
        #y = self.first_activ(y)
        #y = self.second_linear(y)
        return y

In [31]:
loss1 = nn.CrossEntropyLoss()
model1 = LSTM()
model1.to(device)
optimizer1 = torch.optim.Adam(model.parameters(), lr=0.0025)

In [33]:
epochs = 10
for epoch in range(epochs):
    sum_loss = 0
    for x_b, y_b in dl:
        x_b = x_b.to(device)
        y_b = y_b.to(device)
        optimizer1.zero_grad()
        outputs = model1(x_b)
        #outputs= outputs.view(x_b.shape[0],15)
        loss_value = loss1(outputs, y_b)
        loss_value.backward()
        optimizer1.step()
        sum_loss += loss_value
        x_b = x_b.to("cpu")
        y_b = y_b.to("cpu")
        
        #sys.stderr.write(f'Батч {batch + 1}/{len(dl)}, Значение функции потерь: {loss_value.item()}\r')

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {sum_loss/len(dl)}')

Эпоха 1, Значение функции потерь: 2.8378453254699707
Эпоха 2, Значение функции потерь: 2.837843894958496
Эпоха 3, Значение функции потерь: 2.8378055095672607
Эпоха 4, Значение функции потерь: 2.8377745151519775
Эпоха 5, Значение функции потерь: 2.83784556388855
Эпоха 6, Значение функции потерь: 2.8378443717956543
Эпоха 7, Значение функции потерь: 2.8378472328186035
Эпоха 8, Значение функции потерь: 2.837801933288574
Эпоха 9, Значение функции потерь: 2.8378467559814453
Эпоха 10, Значение функции потерь: 2.8378443717956543


In [47]:
torch.cuda.empty_cache() 

In [49]:
answ1 = model1.forward(X_test[:400].to(device))

In [50]:
answ_encoded1 = []
for i in answ1:
    answ_encoded1.append(torch.argmax(i).to("cpu").item())

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test[:400], answ_encoded1))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       242
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         7
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00        14
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         4
          10       0.01      1.00      0.01         2
          11       0.00      0.00      0.00        99
          12       0.00      0.00      0.00         5
          13       0.00      0.00      0.00         5
          14       0.00      0.00      0.00         3

    accuracy                           0.01       400
   macro avg       0.00      0.07      0.00       400
weighted avg       0.00      0.01      0.00       400



c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [21]:
class GRU(nn.Module):
    def __init__(self):
        super(GRU, self).__init__()
        self.rnn = nn.GRU(300, 64, batch_first = True)
        self.first_linear = nn.Linear(64, 15)
        #self.first_linear = nn.Linear(64, 32)
        #self.first_activ = nn.LeakyReLU()
        #self.second_linear = nn.Linear(32, 15)
    def forward(self, x):
        y = self.rnn(x)[0]
        #y = self.first_linear(torch.index_select(y, 1, torch.tensor(y.shape[1]-1).to(device)))
        y = self.first_linear(y.mean(dim=1))
        #y = self.first_activ(y)
        #y = self.second_linear(y)
        return y

In [22]:
loss2 = nn.CrossEntropyLoss()
model2 = GRU()
model2.to(device)
optimizer2 = torch.optim.Adam(model.parameters(), lr=0.0025)

In [23]:
epochs = 10
for epoch in range(epochs):
    batch = 0
    for x_b, y_b in dl:
        x_b = x_b.to(device)
        y_b = y_b.to(device)
        optimizer2.zero_grad()
        batch+=1
        outputs = model2(x_b)
        #outputs= outputs.view(x_b.shape[0],15)
        loss_value = loss2(outputs, y_b)
        loss_value.backward()
        
        optimizer2.step()
        
        #sys.stderr.write(f'Батч {batch + 1}/{len(dl)}, Значение функции потерь: {loss_value.item()}\r')

    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 2.7297275066375732
Эпоха 2, Значение функции потерь: 2.747100591659546
Эпоха 3, Значение функции потерь: 2.670332670211792
Эпоха 4, Значение функции потерь: 2.728611469268799
Эпоха 5, Значение функции потерь: 2.73689866065979
Эпоха 6, Значение функции потерь: 2.7398886680603027
Эпоха 7, Значение функции потерь: 2.6949825286865234
Эпоха 8, Значение функции потерь: 2.7338626384735107
Эпоха 9, Значение функции потерь: 2.722081422805786
Эпоха 10, Значение функции потерь: 2.7357239723205566


In [24]:
answ2 = model2.forward(X_test.to(device))
answ_encoded2 = []
for i in answ2:
    answ_encoded2.append(torch.argmax(i).to("cpu").item())
from sklearn.metrics import classification_report
print(classification_report(y_test, answ_encoded2))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       577
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00        15
           3       0.01      1.00      0.03        14
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00        34
           6       0.00      0.00      0.00        33
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00        19
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         7
          11       0.00      0.00      0.00       239
          12       0.00      0.00      0.00        12
          13       0.00      0.00      0.00        26
          14       0.00      0.00      0.00         5

    accuracy                           0.01       997
   macro avg       0.00      0.07      0.00       997
weighted avg       0.00   

c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\XE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier